In [1]:
import scipy.io
data = scipy.io.loadmat('../../url.mat')

MemoryError: 

**create list for labels and data, where one entry is the data for the day with this index**

In [ ]:
import numpy as np

num_of_days = 120
X, Y = [], []

for i in range(num_of_days):
    day_data = data["Day" + str(i)]
    X.append(day_data[0][0][0])
    Y.append(day_data[0][0][1])

**continous learn classifier**

### single urls

from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(random_state=123)

err = 0
for i in range(1): # looping through days
    
    # change X to row format for faster slicing row-wise.
    X_curr_day, Y_curr_day = X[i].tocsr(), Y[i]
    
    for j in range(X_curr_day.shape[0]): # looping through individual urls
        X_curr_url, Y_curr_url = X_curr_day[j,:], Y_curr_day[j] 
        
        if (j > 0):
            if (clf.predict(X_curr_url) != Y_curr_url):
                err = err + 1
        
        clf.partial_fit(X_curr_url, Y_curr_url, classes=list(range(2))) # Continous fitting of urls and label
print(err)

In [ ]:
### url batches (n=100)
from datetime import datetime
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss='log_loss')
batch_size = 100
err = 0

start = datetime.now().time() # time object
print("start =", start)

for i in range(num_of_days): # looping through days
    
   if (i != 45):
        # change X to row format for faster slicing row-wise.
        X_curr_day = X[i].tocsr()
    
        # split the data in slices of batch_size
        batches_amount = int(X_curr_day.shape[0] / batch_size)
        Y_curr_day = np.array_split(Y[i], batches_amount)
    
        for j in range(batches_amount): # looping through individual urls
            select_ind = np.arange(j * batch_size, (j+1) * batch_size)
        
            X_curr_url_batch, Y_curr_url_batch = X_curr_day[select_ind,:], Y_curr_day[j] 
        
            # flatten y to 1d
            Y_curr_url_batch = Y_curr_url_batch.ravel()
        
            if (j > 0):
                Y_preds = clf.predict(X_curr_url_batch)
            
                for k in range(batch_size):
                    if(Y_preds[k] != Y_curr_url_batch[k]):
                        err = err + 1
        
    
            clf.partial_fit(X_curr_url_batch, Y_curr_url_batch, classes=list(range(2))) # Continous fitting of urls and label
        
        print("Error-rate Day {}   : {}".format(i,err / X_curr_day.shape[0]))
        #print(i)
        #print(err / X_curr_day.shape[0])
        err = 0
end = datetime.now().time() # time object
print("end =", end)
    # batch size 100, first 15 days
    # 0.033125 -> 0.021 (lowest day14 0.0162)
    
    # batch size 100 all days (exc45) -> 6.5min
    # 0.033125 -> 0.01335 (0.0064 96)

**evaluate accuracy**

from sklearn.metrics import accuracy_score

Y_test_preds = []
for j in range(len(X_test)): ## Looping through test batches for making predictions
    Y_preds = clf.predict(X_test[j])
    Y_test_preds.extend(Y_preds.tolist())

# flatten y_test
Y_test = [element for sublist in Y_test for element in sublist]
print("Test Accuracy      : {}".format(accuracy_score(Y_test, Y_test_preds)))